In [ ]:
import zipfile
with zipfile.ZipFile('./data/ReutersNews106521.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/2006-2013')

In [1]:
import pandas as pd
import numpy as np
import pickle
from striprtf.striprtf import rtf_to_text
import string
from os import listdir
from datetime import datetime
from tqdm import tqdm
import cathay.re.SVO_final as SVO
import re
from cathay.config import ApplicationConfig
import boto3
from multiprocessing import Pool
import torch.multiprocessing as mp
import nltk
from transformers import *
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch import optim
import torch.nn.functional as F
# from sklearn.model_selection import train_test_split

/home/nlp/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nlp/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nlp/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nlp/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

# 2006-2013

In [123]:
def Preprocess(dataset):
    datas = []
    for folder in tqdm(dataset):
        files = listdir(path + folder)
        for file in files:
            if file[0] != '.':
                with open(path + folder + '/' + file, 'r') as f:
                    new = f.readlines()

                if new != []:
                    data = {}
                    new = [x.replace('\n', '') for x in new if x[:2] != '\n' and len(x) > 3]
                    data['title'] = new[0].replace('-- ', '')
                    data['source'] = 'Reuters News'
                    date = new[2].replace('-- ', '')
                    data['date'] = datetime.strptime(' '.join(date.split(' ')[:4]), '%a %b %d, %Y').strftime('%Y%m%d')
                    data['text'] = ''.join(new[4:])
                    datas.append(data)
                
    return datas

In [124]:
path = './financial-news-dataset-master/ReutersNews106521/'
folders = [x for x in listdir(path) if len(x) == 8]

n_workers = 5
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(folders) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(folders)
        else:
            batch_end = (len(folders) // n_workers) * (i + 1)

        batch = folders[batch_start: batch_end]
        results[i] = pool.apply_async(Preprocess, [batch])

    pool.close()
    pool.join()

processed = []
for result in results:
    processed += result.get()

100%|██████████| 365/365 [20:46<00:00,  3.42s/it]


In [125]:
len(processed)

69944

In [126]:
path = './data/2006-2013/'
folders = [x for x in listdir(path) if len(x) == 8]

n_workers = 5
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(folders) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(folders)
        else:
            batch_end = (len(folders) // n_workers) * (i + 1)

        batch = folders[batch_start: batch_end]
        results[i] = pool.apply_async(Preprocess, [batch])

    pool.close()
    pool.join()

for result in results:
    processed += result.get()

100%|██████████| 152/152 [15:52<00:00,  6.26s/it]


In [127]:
len(processed)

106494

In [133]:
news = pd.DataFrame({'id':range(len(processed)), 'title':[x['title'] for x in processed], 'date':[x['date'] for x in processed], 'source':[x['source'] for x in processed], 'document_body':[x['text'] for x in processed]})

In [135]:
with open('2006-2013.pkl', 'wb') as f:
    pickle.dump(news, f)

# RTF to DataFrame (2014-2019)

In [3]:
years = ['2014', '2015', '2016', '2017', '2018', '2019']

In [4]:
all_news = []
for year in years:
    mypath = './data/' + year + '/'
    files = listdir(mypath)
    for file in tqdm(files):
        with open(mypath + file, 'r') as f:
            data = f.readlines()
            
        for i in data:
            if i[:4] == '\\par' and rtf_to_text(i) != '\n':
                new = {}
                text = rtf_to_text(i)
                text = text.split('\n')
                text = [x for x in text if x != '']
                if len(text) < 5:
                    continue
                    
                count = 0
                # 總字數
                while '字' not in text[count]:
                    count += 1
                # 作者
                if 'By' in text[count-1] and count-2 >= 0:
                    new['title'] = text[count-2]
                else:
                    new['title'] = text[count-1]
                new['date'] = ''.join([text[count+1].split(' ')[x].zfill(2) for x in [0, 2, 4]])
                new['source'] = text[count+2]
                count = count + 2
                
                while '(c)' not in text[count] and 'Copyright' not in text[count]:
                    count += 1
                if text[count+1][-1] in string.ascii_letters:
                    s = text[count+1] + '.'
                else:
                    s = text[count+1]
                for j in range(count+2, len(text)-2):
                    if text[j][-1] in string.ascii_letters:
                        s += ' ' + text[j] + '.'
                    else:
                        s += ' ' + text[j]
                new['text'] = s
                all_news.append(new)
news = pd.DataFrame({'id':range(len(all_news)), 'title':[x['title'] for x in all_news], 'date':[x['date'] for x in all_news], 'source':[x['source'] for x in all_news], 'document_body':[x['text'] for x in all_news]})

100%|██████████| 97/97 [01:18<00:00,  1.23it/s]


In [148]:
with open('2014-2019.pkl', 'wb') as f:
    pickle.dump(news, f)

# Event Extraction

In [2]:
# with open('2014-2019.pkl', 'rb') as f:
#     news = pickle.load(f)
with open('2006-2013.pkl', 'rb') as f:
    news = pickle.load(f)

In [3]:
news.head()

,id,title,date,source,document_body
0,0,Exxon Mobil offers plan to end Alaska dispute,20061020,Reuters News,"ANCHORAGE, Alaska (Reuters) - Exxon Mobil ( ..."
1,1,"Hey buddy, can you spare $600 for a Google share?",20061020,Reuters News,SAN FRANCISCO/NEW YORK (Reuters) - Wall Stre...
2,2,AOL CEO says sales may shrink for two years -p...,20061021,Reuters News,FRANKFURT (Reuters) - Internet service provi...
3,3,"Fed to keep hawkish tone, hold rates steady",20061022,Reuters News,WASHINGTON (Reuters) - The central bank is e...
4,4,Pluspetrol says losing $2.4 mln/day in Peru pr...,20061021,Reuters News,"LIMA, Peru (Reuters) - Argentine oil company..."


In [4]:
news_dict = news.to_dict('records')

### Title

In [5]:
aws_nlu_config =  ApplicationConfig.get_aws_nlu_config()
comprehend = boto3.client(aws_access_key_id=aws_nlu_config['access_key'], aws_secret_access_key=aws_nlu_config['secret_key'], service_name='comprehend', region_name=aws_nlu_config['region'])

In [6]:
def title_preprocess(sent, comprehend):
    if sent.find('-') == 8 and sent[:6] == 'UPDATE':
        sent = sent[9:]
    if sent.find('-') == 8 and sent[:6] == 'WRAPUP':
        sent = sent[9:]
    if sent.find('-') == 3 and sent[:3] == 'RPT':
        sent = sent[5:]
    while sent.find('-') != -1 and sent[:sent.find('-')].isupper():
        sent = sent[sent.find('-')+1:]
    sent = sent.replace(' - ',' ')
    sent = sent.replace("''",' ')
    sent = re.sub("[+\!\/\\_$%^*()+.:\"“”]+|[+——！，。？、~@#￥%……&*（）：`]+", '', sent)
    sent = sent.replace('\\',' ')
    sent = sent.replace('  ',' ')
    if sent[0] in ['-', ' ']:
        sent = sent[1:]
        
    # 只留句首以及專有名詞大寫
    idx = sent.find(' ')
    # 找專有名詞
    entity = comprehend.detect_entities(Text=sent[idx:], LanguageCode='en')['Entities']
    b = sent[idx:].lower()
    b = b.lower()
    s = ''
    end = 0
    for i in entity:
        s += b[end:i['BeginOffset']]
        s += i['Text']
        end = i['EndOffset']
    s += b[end:]
    sent = sent[:idx] + s
    return sent

In [7]:
# 使用Title，過濾掉 'The Year Ahead' 和 'Quick Takes'
def Event_extrations(dataset):
    processed = []
    for sample in tqdm(dataset):
        sent = re.split(':|;|---', sample['title'])[-1]
        if sent != '' and 'The Year Ahead' not in sent and 'Quick Takes' not in sent:
            tmp = {}
            try:
                sent = title_preprocess(sent, comprehend)
                svo = SVO.SVO(sent)
                svo_result = svo.find_svo()
                tmp['id'] = sample['id']
                tmp['date'] = sample['date']
                tmp['title'] = sample['title']
                tmp['title_SVO'] = svo_result
                processed.append(tmp)
            except:
                print(sent)
    return processed

In [8]:
n_workers = 8
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(news_dict) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(news_dict)
        else:
            batch_end = (len(news_dict) // n_workers) * (i + 1)

        batch = news_dict[batch_start: batch_end]
        results[i] = pool.apply_async(Event_extrations, [batch])

    pool.close()
    pool.join()

processed = []
for result in results:
    processed += result.get()

  3%|▎         | 442/13311 [04:45<2:03:09,  1.74it/s]

Some 4 trillion wiped off world stocks in 2 weeks


 18%|█▊        | 2434/13311 [26:11<2:24:25,  1.26it/s]

SocGen boss survives and says bank can too


 26%|██▋       | 3527/13311 [38:07<1:33:21,  1.75it/s]

...

 67%|██████▋   | 8944/13311 [1:29:47<29:02,  2.51it/s]  

Obama on attack in foreign policy debate, but Romney steady


 97%|█████████▋| 12971/13311 [2:00:06<02:30,  2.25it/s]

US Airways fined 12 million over disabilities infractions


100%|██████████| 13317/13317 [2:05:58<00:00,  1.76it/s]


In [19]:
# with open('news_preprocessed.pkl', 'wb') as f:
#     pickle.dump(processed, f)

### Integrate to SVO

In [21]:
with open('news_preprocessed.pkl', 'rb') as f:
    processed = pickle.load(f)

In [23]:
# 標題的整合
class to_SVO():
    def __init__(self):
        self._be = ['is', 'are', 'am', 'was', 'were']
        
    def to_SVO(self, data):
        if data == 'Sentence can not find SVO.':
            return []
        
        self._data_key = data.keys()
        self._results = []
        # key: main / which..., value: [{}, {}]->dictionary(keys['subject', 'predicate', 'object']
        for key, value in data.items(): 
            for svos in value:                
                # 只有主詞
                if svos['subject'] != [] and svos['predicate'] == [] and svos['object'] == []:
                    self._Subject_only(svos)
                    
                # 沒有受詞                
                if svos['subject'] != [] and svos['predicate'] != [] and svos['object'] == []:
                    self._No_Object(svos)
                
                # 主動受詞都有
                if svos['subject'] != [] and svos['predicate'] != [] and svos['object'] != []:
                    self._Complete(svos)
        
        return self._results
    
    def _Attr_flatten(self, attrs):
        attr_flatten = []
        for attr in [x for x in attrs if isinstance(x, dict) == False and x != None]:
            attr_flatten.append(attr)
        for attr in [x for x in attrs if isinstance(x, dict) == True]:
            for i in ['predicate', 'object']:
                for j in attr[i]:
                    attr_flatten.append(j[0])
                    attr_flatten += self._Attr_flatten(j[1])
        return attr_flatten
    
    def _Subject_only(self, svos):
        # svo: ('', [])
        for svo in svos['subject']:
            # 主詞非dic的Attr
            S_attr = []
            for attr in [x for x in svo[1] if isinstance(x, dict) == False and x != None]:
                S_attr.append(attr)
                
            # 主詞Attr含有動詞，可形成事件
            if True in [isinstance(x, dict) for x in svo[1]]:
                for attr in [x for x in svo[1] if isinstance(x, dict) == True]:
                    if attr['object'] != []:
                        self._results.append([(svo[0], ' '.join(S_attr)), 
                                        (attr['predicate'][0][0], ' '.join(self._Attr_flatten(attr['predicate'][0][1]))), 
                                        (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                    
                    # dictionary沒有object
                    else:
                        self._results.append([(svo[0], ' '.join(S_attr)), 
                                        (attr['predicate'][0][0], ' '.join(self._Attr_flatten(attr['predicate'][0][1])))])
                        
            # 主詞Attr沒有動詞，無法形成事件
            else:
                self._results.append([(svo[0], ' '.join(S_attr))])
    
    def _No_Object(self, svos):
        # 連接詞
        for subject in svos['subject']:
            S = subject[0]
            S_attr = self._Attr_flatten(subject[1])
            for predicate in svos['predicate']:
                P = predicate[0]
                P_attr = []
                for attr in [x for x in predicate[1] if isinstance(x, dict) == False and x != None]:
                    P_attr.append(attr)
                
                # 動詞Attr可以當受詞
                if True in [isinstance(x, dict) for x in predicate[1]]:
                    for attr in [x for x in predicate[1] if isinstance(x, dict) == True]:
                        if 'predicate' in attr.keys() and 'object' in attr.keys():
                            self._results.append([(S, ' '.join(S_attr)), 
                                             (' '.join([P] + [attr['predicate'][0][0]]), ' '.join(P_attr + self._Attr_flatten(attr['predicate'][0][1]))), 
                                             (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                # 動詞Attr不能當受詞
                else:        
                    self._results.append([(S, ' '.join(S_attr)), (P, ' '.join(P_attr))])
    
    def _Complete(self, svos):
        for subject in svos['subject']:
            S = subject[0]
            S_attr = self._Attr_flatten(subject[1])
            for predicate in svos['predicate']:
                P = predicate[0]
                P_attr = self._Attr_flatten(predicate[1])
                for obj in svos['object']:
                    # be動詞 + 受詞是形容詞 + 受詞Attr有dictionary
                    if P in self._be and [x for x in nltk.pos_tag([y for y in obj[0].split(' ') if y != '']) if 'NN' in x[1]] == [] and \
                    True in [isinstance(x, dict) for x in obj[1]]:
                        tmp_P = [P] + [obj[0]]
                        for attr in [x for x in obj[1] if isinstance(x, dict) == False and x != None]:
                            tmp_P.append(attr)
                        for attr in [x for x in obj[1] if isinstance(x, dict) == True]:
                            if 'predicate' in attr.keys() and 'object' in attr.keys():
                                self._results.append([(S, ' '.join(S_attr)), 
                                                     (' '.join(tmp_P + [attr['predicate'][0][0]]), ' '.join(P_attr)), 
                                                     (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                            # attr只有predicate
                            elif 'predicate' in attr.keys():
                                pos = nltk.pos_tag(attr['predicate'][0][0].split(' '))
                                self._results.append([(S, ' '.join(S_attr)), 
                                                     (' '.join(tmp_P + [x[0] for x in pos if 'VB' not in x[1]]), ' '.join(P_attr)), 
                                                     (' '.join([x[0] for x in pos if 'VB' in x[1]]), '')])
                            
                            # 受詞在動詞的Attr中
                            else:
                                for attr in [x for x in predicate[1] if isinstance(x, dict) == True]:
                                    self._results.append([(S, ' '.join(S_attr)), 
                                                         (' '.join(tmp_P + [attr['predicate'][0][0]]), ''), 
                                                         (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                    # 正常狀態
                    else:
                        self._results.append([(S, ' '.join(S_attr)), 
                                             (P, ' '.join(P_attr)), 
                                             (obj[0], ' '.join(self._Attr_flatten(obj[1])))])
                        

In [24]:
def Integrate(dataset):
    processed = []
    for sample in tqdm(dataset):
        tmp = {}
        try:
            tmp['id'] = sample['id']
            tmp['date'] = sample['date']
            tmp['title'] = sample['title']
            tmp['title_SVO'] = sample['title_SVO']
            integrate = to_SVO()
            tmp['integrate_SVO'] = integrate.to_SVO(sample['title_SVO'])
            processed.append(tmp)
        except:
            print(sample['id'])
    return processed

In [25]:
n_workers = 4
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(processed) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(processed)
        else:
            batch_end = (len(processed) // n_workers) * (i + 1)

        batch = processed[batch_start: batch_end]
        results[i] = pool.apply_async(Integrate, [batch])

    pool.close()
    pool.join()

end = []
for result in results:
    end += result.get()

100%|██████████| 26623/26623 [00:00<00:00, 27964.61it/s]


In [34]:
# with open('news_preprocessed_integrate.pkl', 'wb') as f:
#     pickle.dump(data, f)

# Event Embedding

In [35]:
with open('news_preprocessed_integrate_1.pkl', 'rb') as f:
    data = pickle.load(f)

In [36]:
len(data)

106483

In [37]:
len(set([x['date'] for x in data]))

2582

### Word Embedding(Bert)

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

#### Convert to Token

In [ ]:
# train = [item for sublist in [x['integrate_SVO'] for x in data] for item in sublist]

In [39]:
datas = []
for i in tqdm(list(set([x['date'] for x in data]))):
    tmp = {}
    tmp['date'] = i
    tmp['SVO'] = [item for sublist in [x['integrate_SVO'] for x in data if x['date'] == i] for item in sublist]
    datas.append(tmp)

100%|██████████| 2582/2582 [01:17<00:00, 33.15it/s]


In [40]:
def Tokenize(dataset):
    datas = []
    for data in tqdm(dataset):
        tmp = {}
        tmp['date'] = data['date']
        svos = []
        for i in data['SVO']:
            S = tokenizer.encode(i[0][0], add_special_tokens = False)
            S_attr = tokenizer.encode(i[0][1], add_special_tokens = False)

            if len(i) == 1:
                svos.append([(S, S_attr)])
                continue

            if len(i) >= 2:
                P = tokenizer.encode(i[1][0], add_special_tokens = False)
                P_attr = tokenizer.encode(i[1][1], add_special_tokens = False)

                if len(i) == 2:
                    svos.append([(S, S_attr), (P, P_attr)])
                    continue

            if len(i) == 3:
                O = tokenizer.encode(i[2][0], add_special_tokens = False)
                O_attr = tokenizer.encode(i[2][1], add_special_tokens = False)

                svos.append([(S, S_attr), (P, P_attr), (O, O_attr)])
        tmp['SVO'] = svos
        datas.append(tmp)
    return datas

In [41]:
n_workers = 4
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(datas) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(datas)
        else:
            batch_end = (len(datas) // n_workers) * (i + 1)

        batch = datas[batch_start: batch_end]
        results[i] = pool.apply_async(Tokenize, [batch])

    pool.close()
    pool.join()

train_token = []
for result in results:
    train_token += result.get()

100%|██████████| 645/645 [00:15<00:00, 42.71it/s]


In [54]:
len(train_token)

2582

In [53]:
# with open('news_token.pkl', 'wb') as f:
#     pickle.dump(train_token, f)

#### Padding

In [12]:
# def Padding(dataset):
#     datas = []
#     for data in tqdm(dataset):
#         tmp = {}
#         tmp['id'] = data['id']
#         tmp['date'] = data['date']
        
#         processed = []
#         processed_len = []
#         for i in data['title_token']:
#             Subject_len = (len(i[0][0]), len(i[0][1]))
#             S = (pad_to_len(i[0][0], S_len), pad_to_len(i[0][1], S_attr_len))
#             if len(i) == 1:
#                 processed.append([S])
#                 processed_len.append([Subject_len])
            
#             if len(i) >= 2:
#                 Predicate_len = (len(i[1][0]), len(i[1][1]))
#                 P = (pad_to_len(i[1][0], P_len), pad_to_len(i[1][1], P_attr_len))
#                 if len(i) == 2:
#                     processed.append([S, P])
#                     processed_len.append([Subject_len, Predicate_len])
                    
#             if len(i) == 3:
#                 Object_len = (len(i[2][0]), len(i[2][1]))
#                 O = (pad_to_len(i[2][0], O_len), pad_to_len(i[2][1], O_attr_len))
#                 processed.append([S, P, O])
#                 processed_len.append([Subject_len, Predicate_len, Object_len])
            
#         tmp['title_token'] = processed
#         tmp['title_len'] = processed_len
#         datas.append(tmp)
    
#     return datas
    
# def pad_to_len(arr, padded_len):
#     if len(arr) > padded_len:
#         arr = arr[-padded_len:]
#     if len(arr) < padded_len:
#         arr = arr + [0] * (padded_len - len(arr)) 
#     return arr    

In [13]:
# tokens = [item for sublist in [x['title_token'] for x in train_token] for item in sublist]
# S_len = max([len(x[0][0]) for x in tokens])
# S_attr_len = max([len(x[0][1]) for x in tokens])
# P_len = max([len(x[1][0]) for x in tokens if len(x) >= 2])
# P_attr_len = max([len(x[1][1]) for x in tokens if len(x) >= 2])
# O_len = max([len(x[2][0]) for x in tokens if len(x) == 3])
# O_attr_len = max([len(x[2][1]) for x in tokens if len(x) == 3])

In [14]:
# n_workers = 4
# results = [None] * n_workers
# with Pool(processes=n_workers) as pool:
#     for i in range(n_workers):
#         batch_start = (len(train_token) // n_workers) * i
#         if i == n_workers - 1:
#             batch_end = len(train_token)
#         else:
#             batch_end = (len(train_token) // n_workers) * (i + 1)

#         batch = train_token[batch_start: batch_end]
#         results[i] = pool.apply_async(Padding, [batch])

#     pool.close()
#     pool.join()

# train_token = []
# for result in results:
#     train_token += result.get()

100%|██████████| 20284/20284 [00:02<00:00, 8109.33it/s]


In [19]:
# with open('news_token.pkl', 'wb') as f:
#     pickle.dump(train_token, f)

In [16]:
# tokens = [item for sublist in [x['title_token'] for x in train_token] for item in sublist]
# tokens_len = [item for sublist in [x['title_len'] for x in train_token] for item in sublist]
# S = [x[0][0] for x in tokens]
# S_attr = [x[0][1] for x in tokens]
# P = [x[1][0] for x in tokens]
# P_attr = [x[1][1] for x in tokens]
# O = [x[2][0] for x in tokens]
# O_attr = [x[2][1] for x in tokens]

#### Embedding

In [3]:
with open('news_token_1.pkl', 'rb') as f:
    train_token = pickle.load(f)

In [4]:
len(train_token)

2582

In [6]:
def Word_Embedding(token, token_attr):
    with torch.no_grad():
        if token.shape[0] == 0:
            return torch.zeros(768)
        
        a = bert(token.unsqueeze(-1))[0]
        a = a.view(a.shape[0], a.shape[2])
        a = a.mean(0)
        
        # attr空的
        if token_attr.shape[0] == 0:
            return a
        else:
            attr = bert(token_attr.unsqueeze(-1))[0]
            attr = attr.view(attr.shape[0], attr.shape[2])
            attr = attr.mean(0)
            return (a + attr) / 2

In [7]:
train_vector = []
for data in tqdm(train_token):
    tmp = {}
    tmp['date'] = data['date']
    vectors = []
    for token in data['SVO']:
        try:
            S = Word_Embedding(torch.tensor(token[0][0]), torch.tensor(token[0][1]))
            if len(token) == 1:
                vectors.append(torch.stack((S, torch.zeros(768), torch.zeros(768))))

            if len(token) >= 2:
                P = Word_Embedding(torch.tensor(token[1][0]), torch.tensor(token[1][1]))
                if len(token) == 2:
                    vectors.append(torch.stack((S, P, torch.zeros(768))))

            if len(token) == 3:
                O = Word_Embedding(torch.tensor(token[2][0]), torch.tensor(token[2][1]))
                vectors.append(torch.stack((S, P, O)))
        except:
            print(token)
    tmp['SVO'] = vectors
    train_vector.append(tmp)

100%|██████████| 2582/2582 [3:10:47<00:00,  4.43s/it]  


In [13]:
len(train_vector1)

4695

In [16]:
# with open('news_embedding.pkl', 'wb') as f:
#     pickle.dump(train_vector, f)

### Train AutoEncoder

In [171]:
with open('news_embedding.pkl', 'rb') as f:
    train_vector = pickle.load(f)

In [172]:
train_vector[0]

{'date': '20081020',
 'SVO': [tensor([[-0.1546, -0.0797, -0.1861,  ...,  0.3550,  0.9645,  0.3927],
          [-1.7465,  0.3319, -0.1670,  ...,  0.4049,  0.2178,  0.5997],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([[-0.1546, -0.0797, -0.1861,  ...,  0.3550,  0.9645,  0.3927],
          [ 0.5713,  0.5175, -0.1654,  ..., -0.7595, -0.0360, -0.2538],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([[ 0.4560,  0.0729, -0.4603,  ...,  0.0803,  0.5277, -0.3437],
          [-0.8518, -0.5266, -0.7449,  ...,  0.0182,  0.6395,  0.0597],
          [ 0.3278,  0.1609,  0.6035,  ..., -0.3031, -0.0770,  0.2254]]),
  tensor([[-0.3748,  0.3513, -0.2386,  ..., -0.0119,  0.5345,  0.3533],
          [-0.2259, -0.2642,  0.1893,  ...,  0.3353,  0.4613,  0.2331],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([[ 6.6826e-02,  2.7191e-01,  4.3250e-02,  ..., -2.2703e-01,
            2.7248e-01,  1.2428

In [3]:
train = [item for sublist in [x['SVO'] for x in train_vector] for item in sublist]

In [4]:
len(train)

224011

In [10]:
train_dataloader = DataLoader(train, batch_size=256, shuffle=True)

In [11]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)), # 128, 4, 768
            nn.Conv2d(128, 32, (3,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.ConvTranspose2d(32, 128, (3,1), stride=(1,1), padding=(0,0)), # 128, 4, 768
            nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded
        

In [12]:
use_gpu = torch.cuda.is_available()
autoencoder = AutoEncoder()
if use_gpu:
    autoencoder.cuda()

In [13]:
criteria = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
EPOCH = 1

for epoch in range(EPOCH):
    cumulate_loss = 0
    for idx, x in tqdm(enumerate(train_dataloader)):
        x = x.unsqueeze(1)
        if use_gpu:
            x = x.cuda()
        latent, reconstruct = autoencoder(x)
        loss = criteria(reconstruct, x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cumulate_loss += loss.item() * x.shape[0]
        if (idx % 100) == 0:
            print(loss)
            
    print(f'Epoch { "%03d" % epoch }: Loss : { "%.5f" % (cumulate_loss / len(train))}')

1it [00:01,  1.76s/it]

tensor(0.7944, device='cuda:0', grad_fn=<MseLossBackward>)


101it [01:44,  1.04s/it]

tensor(0.0642, device='cuda:0', grad_fn=<MseLossBackward>)


201it [03:28,  1.03s/it]

...

tensor(0.0622, device='cuda:0', grad_fn=<MseLossBackward>)


801it [13:50,  1.03s/it]

tensor(0.0658, device='cuda:0', grad_fn=<MseLossBackward>)


876it [15:06,  1.04s/it]

Epoch 000: Loss : 0.06877


In [14]:
checkpoint_path = 'autoencoder(1channel)_{}.pth'.format(epoch+1) 
torch.save(autoencoder.state_dict(), checkpoint_path)
print('model saved to %s' % checkpoint_path)

model saved to autoencoder(1channel)_1.pth


#### AutoEncoder Validation

# Deep Learning

#### Day-Vector

In [52]:
with open('news_embedding.pkl', 'rb') as f:
    data = pickle.load(f)

In [53]:
class Day_vector():
    def __init__(self, input_path, model_path, channel):
        self.channel = channel
        with open(input_path, 'rb') as f:
            self.data = pickle.load(f)
            
        self.autoencoder = AutoEncoder(self.channel)
        self.autoencoder.load_state_dict(torch.load(model_path))
        self.use_gpu = torch.cuda.is_available()
        if self.use_gpu:
            self.autoencoder.cuda()
    
    def DayVector(self, date):
        self._vector = [x['SVO'] for x in self.data if x['date'] == date][0]
        
        if len(self._vector) > 0:
            # Event Embedding
            with torch.no_grad():
                dataloader = DataLoader(self._vector, batch_size=len(self._vector), shuffle=False)
                for x in dataloader:
                    x = x.unsqueeze(1)
                    if self.use_gpu:
                        x = x.cuda()
                    latent, reconstruct = self.autoencoder(x)

                latent = latent.cpu().detach().numpy()
                latent = latent.reshape(len(latent), 768*self.channel)

            return latent.mean(0)
        else:
            return []

class AutoEncoder(nn.Module):
    def __init__(self, channel):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)), # 128, 4, 768
            nn.Conv2d(128, 32, (3,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.Conv2d(32, channel, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(channel, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.ConvTranspose2d(32, 128, (3,1), stride=(1,1), padding=(0,0)), # 128, 4, 768
            nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded

In [55]:
day_vector = Day_vector('news_embedding.pkl', 'autoencoder(1channel)_1.pth', 1)

In [56]:
date = list(set([x['date'] for x in data]))

In [57]:
datas = {}
for i in tqdm(date):
    day = day_vector.DayVector(i)
    if day != []:
        datas[i] = day
datas = dict([(k,datas[k]) for k in sorted(datas.keys())])

  0%|          | 0/4695 [00:00<?, ?it/s]/home/nlp/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.
100%|██████████| 4695/4695 [10:09<00:00,  7.71it/s]


In [58]:
len(datas)

4686

In [59]:
with open('Event_embedding(1channel).pkl', 'wb') as f:
    pickle.dump(datas, f)

In [25]:
datas

{'20061020': array([ 0.8386301 ,  0.21573278, -0.2367236 , ..., -0.29065183,
         0.14035659,  0.3385951 ], dtype=float32),
 '20061021': array([ 1.3698486 ,  0.28323483,  0.227392  , ...,  0.2282753 ,
        -0.10364173,  0.21362947], dtype=float32),
 '20061022': array([ 1.3954376 ,  0.29485977,  0.6374901 , ...,  0.10777788,
         0.5106691 , -0.48019662], dtype=float32),
 '20061023': array([ 1.148318  ,  0.08844635,  0.6023744 , ..., -0.02133161,
         0.19196652,  0.04699217], dtype=float32),
 '20061024': array([ 1.1130054 ,  0.19780375,  0.6451171 , ..., -0.1090382 ,
         0.1252562 , -0.04977267], dtype=float32),
 '20061025': array([ 1.0501871 ,  0.24997556,  0.55287844, ..., -0.00435719,
         0.02796607, -0.13346355], dtype=float32),
 '20061026': array([ 1.0754712 ,  0.18206319,  0.55892545, ..., -0.05918308,
        -0.08458969, -0.17107256], dtype=float32),
 '20061027': array([ 0.9696972 ,  0.13743287,  0.4794212 , ..., -0.08248325,
         0.10772131, -0.042

### TWII Price
#### 2006/10/20前、2013/11/20之後的資料拔掉

In [60]:
price = pd.read_csv('^TWII.csv')
price['Date'] = price['Date'].apply(lambda x:x.replace('-', ''))
price['return'] = price.shift(-1)['Adj Close'] / price['Adj Close']
price = price.dropna()

In [61]:
price['label'] = 0
price.loc[price[price['return'] > 1].index, 'label'] = 1
# price.loc[price[price['return'] >= 1.001].index, 'label'] = 1
# price.loc[price[price['return'] <= 0.999].index, 'label'] = -1
price = price[price['Date'] <= '20191130']
price = price[price['Date'] >= '20061120']
price = pd.concat((price[price['Date'] <= '20131120'], price[price['Date'] >= '20140101']))
price = price.reset_index(drop=True)

In [62]:
from collections import Counter
a = Counter(price['label'])
a

Counter({1: 1696, 0: 1459})

#### Data split (日期切分) 

In [63]:
with open('Event_embedding(1channel).pkl', 'rb') as f:
    datas = pickle.load(f)

In [64]:
event = [(k, datas[k]) for k in sorted(datas.keys())]
dates = [x[0] for x in event]
event_embedding = [x[1] for x in event]

In [65]:
all_data = []
for i in tqdm(range(len(price))):
    data = {}
    if price.loc[i, 'Date'] in dates:
        data['date'] = dates.index(price.loc[i, 'Date'])
    tmp = int(price.loc[i, 'Date'])
    while str(tmp) not in dates:
        tmp -= 1
    data['date'] = dates.index(str(tmp))
    data['label'] = price.loc[i, 'label']
    all_data.append(data)

100%|██████████| 3155/3155 [00:01<00:00, 2804.34it/s]


In [66]:
test = all_data[-int(len(all_data)*0.1):]
tmp = all_data[:int(len(all_data)*0.9)+1]
valid = tmp[-int(len(tmp)*0.1):]
train = tmp[:int(len(tmp)*0.9)]
print('Train: ', len(train))
print('Valid:' , len(valid))
print('Test: ', len(test))

Train:  2556
Valid: 284
Test:  315


In [173]:
train

[{'date': 31, 'label': 1},
 {'date': 32, 'label': 1},
 {'date': 33, 'label': 1},
 {'date': 34, 'label': 1},
 {'date': 35, 'label': 1},
 {'date': 38, 'label': 0},
 {'date': 39, 'label': 1},
 {'date': 40, 'label': 1},
 {'date': 41, 'label': 1},
 {'date': 42, 'label': 1},
 {'date': 45, 'label': 0},
 {'date': 46, 'label': 1},
 {'date': 47, 'label': 0},
 {'date': 48, 'label': 0},
 {'date': 49, 'label': 0},
 {'date': 52, 'label': 0},
 {'date': 53, 'label': 0},
 {'date': 54, 'label': 1},
 {'date': 55, 'label': 1},
 {'date': 56, 'label': 1},
 {'date': 59, 'label': 0},
 {'date': 60, 'label': 1},
 {'date': 61, 'label': 0},
 {'date': 62, 'label': 1},
 {'date': 63, 'label': 0},
 {'date': 66, 'label': 1},
 {'date': 67, 'label': 1},
 {'date': 68, 'label': 0},
 {'date': 69, 'label': 1},
 {'date': 70, 'label': 1},
 {'date': 74, 'label': 0},
 {'date': 75, 'label': 1},
 {'date': 76, 'label': 0},
 {'date': 77, 'label': 0},
 {'date': 80, 'label': 1},
 {'date': 81, 'label': 0},
 {'date': 82, 'label': 0},
 

## Model

In [129]:
class EventDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return dict(self.data[index])
    
    def collate_fn(self, datas):
        batch = {}
        
        batch['token'] = torch.tensor([list(range(x['date'] - 19, x['date'] + 1)) for x in datas])
#         batch['label'] = torch.tensor([self._label(x['label']) for x in datas])
        batch['label'] = torch.tensor([x['label'] for x in datas])
        
        return batch
    
    def _label(self, label):
        if label == -1:
            return 0
        elif label == 0:
            return 1
        else:
            return 2


In [148]:
train_set = EventDataset(train)
train_loader = DataLoader(train_set, collate_fn=train_set.collate_fn, batch_size=32, shuffle=True)
valid_set = EventDataset(valid)
valid_loader = DataLoader(valid_set, collate_fn=valid_set.collate_fn, batch_size=32, shuffle=True)

In [168]:
class LSTMNet(nn.Module):
    def __init__(self, pretrained_embedding):
        super(LSTMNet, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)), # 128, 4, 768
            nn.Conv2d(128, 32, (3,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.ConvTranspose2d(32, 128, (3,1), stride=(1,1), padding=(0,0)), # 128, 4, 768
            nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
        )
        
        pretrained_embedding = torch.FloatTensor(pretrained_embedding)
        self.embedding = nn.Embedding(
            pretrained_embedding.size(0),
            pretrained_embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(pretrained_embedding)
#         self.embedding.weight.requires_grad = False
        
        bi = True
        self.lstm = nn.LSTM(pretrained_embedding.size(1), 800, 2, dropout=0.2, bidirectional=bi)
        self.hidden2out = nn.Sequential(
            nn.BatchNorm1d(800 * (1+bi) * 4),
            nn.LeakyReLU(0.2),
            nn.Linear(800 * (1+bi) * 4, 400),
            
            nn.BatchNorm1d(400),
            nn.LeakyReLU(0.2),
            nn.Linear(400, 100),
            
            nn.BatchNorm1d(100),
            nn.LeakyReLU(0.2),
            nn.Linear(100, 2))
    
    def forward(self, event):
        x = self.embedding(event)
        out, (_, _) = self.lstm(x)
        pool = F.avg_pool1d(out.transpose(1, 2), 5)
        pool = pool.view(pool.shape[0], pool.shape[1]*pool.shape[2])
#         out = out.mean(1)
        
        y = self.hidden2out(pool)
        return y

In [169]:
EPOCH = 20
model = LSTMNet(event_embedding)
use_gpu = torch.cuda.is_available()
if use_gpu:
    model.cuda()

In [170]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

acc_history = []
loss_history = []
for epoch in range(EPOCH):
    model.train()
    train_loss = []
    train_acc = []
    for data in train_loader:
        if use_gpu:
            event = data['token'].cuda()
            labels = data['label'].cuda()
            
        optimizer.zero_grad()
        output = model(event)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        
        predict = output.max(1)[1]
        acc = np.mean((labels == predict).cpu().numpy())
        train_acc.append(acc)
        train_loss.append(loss.item())
        
    print("Epoch: {}, train Loss: {:.4f}, train accuracy: {:.4f}".format(epoch + 1, np.mean(train_loss), np.mean(train_acc)))
    acc_history.append(np.mean(train_acc))
    loss_history.append(np.mean(train_loss))
    
    model.eval()
    with torch.no_grad():
        valid_acc = []
        for data in valid_loader:
            if use_gpu:
                event = data['token'].cuda()
                labels = data['label'].cuda()

            output = model(event)
            predict = output.max(1)[1]
            acc = np.mean((labels == predict).cpu().numpy())
            valid_acc.append(acc)
        print("Epoch: {}, valid accuracy: {:.4f}".format(epoch + 1, np.mean(valid_acc)))


Epoch: 1, train Loss: 0.7117, train accuracy: 0.4967
Epoch: 1, valid accuracy: 0.5288
Epoch: 2, train Loss: 0.6984, train accuracy: 0.5155
Epoch: 2, valid accuracy: 0.5104
Epoch: 3, train Loss: 0.6959, train accuracy: 0.5066
Epoch: 3, valid accuracy: 0.5198
Epoch: 4, train Loss: 0.6942, train accuracy: 0.5185
Epoch: 4, valid accuracy: 0.5491
Epoch: 5, train Loss: 0.6936, train accuracy: 0.5330
Epoch: 5, valid accuracy: 0.5228
Epoch: 6, train Loss: 0.6892, train accuracy: 0.5321
Epoch: 6, valid accuracy: 0.4826
Epoch: 7, train Loss: 0.6894, train accuracy: 0.5440
Epoch: 7, valid accuracy: 0.5198
Epoch: 8, train Loss: 0.6838, train accuracy: 0.5509
Epoch: 8, valid accuracy: 0.5134
Epoch: 9, train Loss: 0.6792, train accuracy: 0.5667
Epoch: 9, valid accuracy: 0.4573
Epoch: 10, train Loss: 0.6583, train accuracy: 0.5983
Epoch: 10, valid accuracy: 0.5094
Epoch: 11, train Loss: 0.6095, train accuracy: 0.6786
Epoch: 11, valid accuracy: 0.5005
Epoch: 12, train Loss: 0.5559, train accuracy: 0.7

In [157]:
checkpoint_path = 'LSTM_{}.pth'.format(epoch+1) 
torch.save(model.state_dict(), checkpoint_path)
print('model saved to %s' % checkpoint_path)

model saved to LSTM_20.pth


In [162]:
predict

tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1], device='cuda:0')

In [163]:
labels

tensor([0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 0], device='cuda:0')

In [164]:
output

tensor([[-1.6949,  1.8634],
        [-2.2856,  2.2333],
        [ 0.1700, -0.1623],
        [-2.0922,  2.1261],
        [-2.5890,  2.4851],
        [-1.1441,  1.0982],
        [ 1.6853, -1.8341],
        [-2.3723,  2.3570],
        [-2.1275,  2.1507],
        [-2.4942,  2.4654],
        [-0.3400,  0.3147],
        [ 1.0449, -1.0860],
        [-2.9948,  2.9257],
        [-1.6614,  1.5719],
        [-1.0900,  1.0533],
        [-0.9687,  0.9299],
        [ 1.0281, -1.0865],
        [-1.3706,  1.2688],
        [-0.7765,  0.7263],
        [-1.2935,  1.2563],
        [ 1.4716, -1.5658],
        [-1.3502,  1.2849],
        [-1.9837,  1.8963],
        [-2.1241,  2.1631],
        [-1.2080,  1.1664],
        [-2.0760,  2.0435],
        [-1.9521,  1.9695],
        [-1.0206,  1.1023]], device='cuda:0')

In [13]:
for data in train_loader:
    break

In [36]:
pretrained_embedding = torch.FloatTensor(event_embedding)
embedding = nn.Embedding(
    pretrained_embedding.size(0),
    pretrained_embedding.size(1))
embedding.weight = torch.nn.Parameter(pretrained_embedding)

bi = False
lstm = nn.LSTM(pretrained_embedding.size(1), 800, 2, dropout=0.2, bidirectional=bi)
hidden2out = nn.Sequential(
            nn.BatchNorm1d(800 * (1+bi) * 5),
            nn.ReLU(),
            nn.Linear(800 * (1+bi) * 5, 800 * (1+bi)),
            nn.BatchNorm1d(800 * (1+bi)),
            nn.ReLU(),
            nn.Linear(800 * (1+bi), 2))

In [16]:
x = embedding(data['token'])
out, (_, _) = lstm(x)

In [30]:
pool = F.adaptive_max_pool2d(out.transpose(1, 2), (out.shape[2], 5))

In [35]:
pool = pool.view(pool.shape[0], pool.shape[1]*pool.shape[2])

In [43]:
output

tensor([[ 1.3948, -0.7978],
        [-0.0233,  0.0044],
        [-0.1252,  0.0603],
        [-0.1083,  0.0499],
        [-0.1054,  0.0477],
        [-0.1045,  0.0468],
        [-0.1046,  0.0470],
        [-0.1047,  0.0470],
        [-0.1047,  0.0470],
        [-0.1048,  0.0471],
        [-0.1048,  0.0473],
        [-0.1047,  0.0474],
        [-0.1048,  0.0477],
        [-0.1047,  0.0478],
        [-0.1048,  0.0480],
        [-0.1044,  0.0478],
        [-0.1042,  0.0477],
        [-0.1041,  0.0476],
        [-0.1042,  0.0477],
        [-0.1042,  0.0478],
        [-0.1042,  0.0478],
        [-0.1046,  0.0481],
        [-0.1046,  0.0481],
        [-0.1045,  0.0481],
        [-0.1045,  0.0480],
        [-0.1044,  0.0480],
        [-0.1041,  0.0478],
        [-0.1042,  0.0479]], device='cuda:0')